In [15]:
import os
import math
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from tensorflow import keras

In [8]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']

def get_file_list(root_dir):
    file_list = []
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
    return file_list

In [10]:
root_dir = './cat_train_images'
filenames = sorted(get_file_list(root_dir))

TRAIN_SAMPLES = len(filenames)
NUM_CLASSES = 67
IMG_WIDTH, IMG_HEIGHT = 224, 224
batch_size=64

In [11]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  zoom_range=0.2)

In [12]:
train_generator = train_datagen.flow_from_directory(root_dir,
                                                   target_size=(IMG_WIDTH,
                                                               IMG_HEIGHT),
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   seed=12345,
                                                   class_mode='categorical')

Found 126599 images belonging to 67 classes.


In [13]:
model_finetuned = keras.models.load_model("./model-finetuned-210102.h5")
model_finetuned.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                131136    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 67)                4355      
Total params: 23,723,203
Trainable params: 135,491
Non-trainable params: 23,587,712
_________________________________________

In [16]:
model_finetuned.compile(loss='categorical_crossentropy',
                       optimizer=tf.keras.optimizers.Adam(0.001),
                       metrics=['acc'])

model_finetuned.fit_generator(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / batch_size),
    epochs=1
)

Instructions for updating:
Please use Model.fit, which supports generators.
  18/1979 [..............................] - ETA: 2:07:45 - loss: 2.3161 - acc: 0.4288

KeyboardInterrupt: 

In [ ]:
model_finetuned.save("./model-finetuned-epoch2.h5")